In [ ]:
# Using Common crawl index to find the links of required domain

In [2]:
import requests
import argparse
import time
import json
import StringIO
import gzip
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
from urllib2 import URLError, HTTPError
import socket

In [11]:
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [9]:
# cnn.com
# finance.yahoo.com

domain = 'wsj.com'
index_list = ["2019-04", "2019-09", "2019-13"]

In [10]:
def search_domain(domain):
    record_list = []
    print "[*] Trying target domain: %s" % domain
    
    for index in index_list:
        print "[*] Trying index %s" % index
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            records = response.content.splitlines()
            for record in records:
                record_list.append(json.loads(record))  
            print "[*] Added %d results." % len(records)
    print "[*] Found a total of %d hits." % len(record_list)
    return record_list 

In [11]:
record_list = search_domain(domain)

[*] Trying target domain: wsj.com
[*] Trying index 2019-09
[*] Added 12071 results.
[*] Trying index 2019-13
[*] Added 13926 results.
[*] Found a total of 25997 hits.


In [12]:
for record in record_list:
    f = open('links.txt', 'ab')
    f.write(record['url']+'\n')
    f.close()

In [1]:
# Reaad the saved lines

In [13]:
links_file = open('links.txt', 'r')
links = links_file.readlines()
links_file.close()

In [14]:
len(links)

25997

In [5]:
import ssl

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


In [15]:
# Saves the data collected for each keyword in separate text files

def save_keyword_data(keyword, text):
    text = text.encode('utf-8')
    path = keyword+".txt"
    text_file = open(path, "ab")
    text_file.write(text)
    text_file.close()

In [16]:
amazonCount = appleCount = facebookCount = microsoftCount = googleCount = 0

In [ ]:
for url in links:
    url = url.strip()
    try:
        html = urllib2.urlopen(url, context = ctx, timeout = 5).read()
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        for a in ["h1","h2"]:
            for headlines in soup.find_all(a):
                
                #print("reading headlines")
                #headlines = headlines.encode('utf-8')
                
                headlines = headlines.text.strip()
                headlines = headlines.encode('ascii','ignore')
                headlines = headlines.lower()
                
                text = soup.get_text()
                if('apple' in headlines):
                    appleCount+=1
                    save_keyword_data('uber', text)

                if('facebook' in headlines):
                    facebookCount+=1
                    save_keyword_data('facebook', text)
                    
                if('google' in headlines):
                    googleCount+=1
                    save_keyword_data('google', text)
                    
                if('amazon' in headlines):
                    amazonCount+=1
                    save_keyword_data('amazon', text)
                    
                if('microsoft' in headlines):
                    microsoftCount+=1
                    save_keyword_data('microsoft', text)
                    
    except Exception , e:
        print("error", e)
        

In [ ]:
print(amazonCount, appleCount, facebookCount, microsoftCount, googleCount)